In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px



In [14]:
df = pd.read_csv('Planilha/house_prices.csv') 

FileNotFoundError: [Errno 2] No such file or directory: 'Planilha/house_prices.csv'

In [11]:
df.isnull().sum()

NameError: name 'df' is not defined

In [ ]:
def extrair(texto):
    if isinstance(texto, str) and len(texto) >= 19:
        return texto[11:19]
    else:
        return '00:00:00'
df['Horario'] = df['datetime'].apply(extrair)
del df['datetime']


In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].dt.strftime('%d/%m/%Y')
df = df.rename(columns={'date':'Data', 'cash_type':'Pagamento', 'money': 'Valor', 'coffee_name':'Bebida', 'card':'Cartao'})

In [ ]:
mapping = {
    'Tea': '1',
    'Espresso': '2', 
    'Chocolate with coffee': '3', 
    'Chocolate with milk': '4',
    'Coffee with Irish whiskey': '5', 
    'Irish whiskey with milk': '6',
    'Double Irish whiskey': '7', 
    'Double espresso': '8', 
    'Americano with milk': '9',
    'Cappuccino': '10', 
    'Double espresso with milk': '11',
    'Caramel with Irish whiskey': '12', 
    'Latte': '13', 
    'Americano': '14',
    'Super chocolate': '15', 
    'Double vanilla': '16', 
    'Mochaccino': '17',
    'Caramel with chocolate': '18', 
    'Irish whiskey': '19', 
    'Caramel coffee': '20',
    'Vanilla with Irish whiskey': '21', 
    'Chocolate': '22', 
    'Caramel with milk': '23',
    'Coffee with chocolate': '24', 
    'Irish with chocolate': '25', 
    'Vanilla coffee': '26',
    'Double ristretto': '27', 
    'Hot milkshake': '28', 
    'Double chocolate': '29', 
    'Caramel': '30',
    'Hot Chocolate': '31', 
    'Americano with Milk': '32', 
    'Cocoa': '33', 
    'Cortado': '34'
}
df['NumeroBebida'] = df['Bebida'].astype(str)
df['NumeroBebida'] = df['NumeroBebida'].map(mapping)
df['NumeroBebida'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34'],
      dtype=object)

In [ ]:
df = df[['Data', 'Horario', 'Bebida', 'NumeroBebida', 'Valor', 'Pagamento', 'Cartao']]
df

,Data,Horario,Bebida,NumeroBebida,Valor,Pagamento,Cartao
0,08/02/2025,14:26:04,Tea,1,15.00,cash,NaN
1,08/02/2025,14:28:26,Tea,1,15.00,cash,NaN
2,08/02/2025,14:33:04,Espresso,2,20.00,card,NaN
3,08/02/2025,15:51:04,Chocolate with coffee,3,30.00,card,NaN
4,08/02/2025,16:35:01,Chocolate with milk,4,27.00,cash,NaN
...,...,...,...,...,...,...,...
3631,23/03/2025,10:34:54,Cappuccino,10,35.76,card,ANON-0000-0000-1158
3632,23/03/2025,14:43:37,Cocoa,33,35.76,card,ANON-0000-0000-1315
3633,23/03/2025,14:44:16,Cocoa,33,35.76,card,ANON-0000-0000-1315
3634,23/03/2025,15:47:28,Americano,14,25.96,card,ANON-0000-0000-1316


In [ ]:
value = df[(df['Cartao'] == null), (df['Pagamento'] == 'card')]
value

NameError: name 'df' is not defined